# Explore ERDDAP timeseries data using Jupyter Widgets
Inspired by [Jason Grout's excellent ESIP Tech Dive talk on "Jupyter Widgets"](https://youtu.be/CVcrTRQkTxo?t=2596), this notebook uses the `ipyleaflet` and `bqplot` widgets
to interactively explore the last two weeks of time series data from an ERDDAP Server. Select a `standard_name` from the list, then click a station to see the time series.  

In [ ]:
import pandas as pd

`pendulum` is a drop-in replacement for `datetime`, with more useful functions

In [ ]:
import pendulum

`ipyleaflet` and `bqplot` are both Jupyter widgets, so can interact with Python like any other widget.  Since we want to click on a map in a notebook and get an interactive time series plot, they are perfect tools to use here. 

In [ ]:
import ipyleaflet as ipyl
import bqplot as bq
import ipywidgets as ipyw

To make working with ERDDAP simpler, we use `erddapy`, a high-level python interface to ERDDAP's RESTful API

In [ ]:
from erddapy import ERDDAP

This code should work with minor modifications on any ERDDAP (v1.64+) endpoint that has `cdm_data_type=timeseries` datasets.  Here we use the US-IOOS [NERACOOS ERDDAP server](http://www.neracoos.org/erddap/index.html). 

In [ ]:
endpoint = 'http://www.neracoos.org/erddap'

Change these initial values for other ERDDAP endpoints or regions of interest

In [ ]:
standard_name = 'significant_height_of_wind_and_swell_waves'
dataset = 'B01_accelerometer_all'
center = [42.5, -68]
zoom = 6

Here we get find datasets with data in the last two weeks and then plot that data, but this could be made more flexible, perhaps with widgets controlling the time range to plot and search.

In [ ]:
now = pendulum.utcnow()
max_time = now
min_time = now.subtract(weeks=2) 

In [ ]:
kwargs = {'time%3E=': min_time, 'time%3C=': max_time}

In [ ]:
e = ERDDAP(server_url=endpoint)

Find all the `standard_name` attributes that exist on this ERDDAP endpoint, using [ERDDAP's "categorize" service](http://www.neracoos.org/erddap/categorize/index.html)

In [ ]:
url='{}/categorize/standard_name/index.csv'.format(endpoint)
df = pd.read_csv(url, skiprows=[1, 2])
vars = df['Category'].values

Create a dropdown menu widget with all the `standard_name` values found

In [ ]:
dpdown = ipyw.Dropdown(options=vars, value=standard_name)

This function convert an ERDDAP timeseries CSV response to a Pandas dataframe

In [ ]:
def download_csv(url):
    return pd.read_csv(url, index_col='time', parse_dates=True, skiprows=[1])

This function puts lon,lat and datasetID into a GeoJSON feature

In [ ]:
def point(dataset,lon,lat):

    geojsonFeature = {
        "type": "Feature",
        "properties": {
            "datasetID": dataset,
            "short_dataset_name": dataset[:3]
        },
        "geometry": {
            "type": "Point",
            "coordinates": [lon, lat]
        }
    };
    geojsonFeature['properties']['style'] = {'color': 'Grey'}
    return geojsonFeature

This function returns lon/lat values for a specified `datasetID` using the `allDatasets` dataset, available on version 1.64 and higher.

In [ ]:
def lonlats(e, datasetID):
    url='{}/tabledap/allDatasets.csv?datasetID%2CminLongitude%2CminLatitude&datasetID=%22{}%22'.format(e.server_url, datasetID)
    df = pd.read_csv(url, skiprows=[1])
    lon = df['minLongitude'].values[0]
    lat = df['minLatitude'].values[0]
    return lon,lat

This function finds all the datasets with a given standard_name in the specified time period, and return GeoJSON

In [ ]:
def stdname2geojson(e, standard_name, min_time, max_time):
    '''return geojson containing lon, lat and datasetID for all matching stations'''
    search_url = e.get_search_url(response='csv', cdm_data_type='timeseries', 
                                  standard_name=standard_name, min_time=min_time, max_time=max_time)
    dfs = pd.read_csv(search_url)
    datasets = dfs['Dataset ID'].values

    feature_list=[]
    for dataset in datasets:
        lon, lat = lonlats(e, dataset)
        geojsonFeature = point(dataset, lon, lat)
        feature_list.append(geojsonFeature)

    data = {'features':feature_list}
    return data

This function updates the time series plot when a station marker is clicked

In [ ]:
def click_handler(event=None, id=None, properties=None):
    datasetID = properties['datasetID']
    kwargs = {'time%3E=': min_time, 'time%3C=': max_time}
    df, var = get_data(datasetID, dpdown.value, kwargs)
    figure.marks[0].x = df.index
    figure.marks[0].y = df[var]
    figure.title = '{} - {}'.format(properties['short_dataset_name'], var)

This function updates the map when a new variable is selected

In [ ]:
def update_dpdown(change):
    standard_name = change['new']
    data = stdname2geojson(e, standard_name, min_time, max_time)
    feature_layer = ipyl.GeoJSON(data=data)
    feature_layer.on_click(click_handler)
    map.layers = [map.layers[0], feature_layer]

This specifies which function to use when a variable is selected from the dropdown list

In [ ]:
dpdown.observe(update_dpdown, names=['value'])

This function returns the specified dataset time series values as a Pandas dataframe

In [ ]:
def get_data(dataset, standard_name, kwargs):
    var = e.get_var_by_attr(dataset_id=dataset, standard_name=standard_name)[0]
    download_url = e.get_download_url(dataset_id=dataset, 
                                  variables=['time',var], response='csv', **kwargs)
    df = download_csv(download_url)
    return df, var

This defines the initial `ipyleaflet` map 

In [ ]:
map = ipyl.Map(center=center, zoom=zoom, layout=ipyl.Layout(width='650px', height='350px'))
data = stdname2geojson(e, standard_name, min_time, max_time)
feature_layer = ipyl.GeoJSON(data=data)
feature_layer.on_click(click_handler)
map.layers = [map.layers[0], feature_layer]

This defines the intitial `bqplot` time series plot

In [ ]:
dt_x = bq.DateScale()
sc_y = bq.LinearScale()

df, var = get_data(dataset, standard_name, kwargs)
time_series = bq.Lines(x=df.index, y=df[var], scales={'x': dt_x, 'y': sc_y})
ax_x = bq.Axis(scale=dt_x, label='Time')
ax_y = bq.Axis(scale=sc_y, orientation='vertical')
figure = bq.Figure(marks=[time_series], axes=[ax_x, ax_y])
figure.title = '{} - {}'.format(dataset[:3], var)
figure.layout.height = '300px'
figure.layout.width = '800px'

This specifies the widget layout

In [ ]:
ipyw.VBox([dpdown, map, figure])